In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense,GlobalAveragePooling2D, Flatten, UpSampling2D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
#Load dataset
(X_train, Y_train), (X_test, Y_test) = datasets.cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [ ]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
restnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
x = tf.keras.layers.Input(shape=(32,32,3))
resize = tf.keras.layers.UpSampling2D(size=(7,7))(x)
feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(resize)
x = tf.keras.layers.GlobalAveragePooling2D()(feature_extractor)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
x = tf.keras.layers.Dense(512, activation="relu")(x)
x = tf.keras.layers.Dense(10, activation="softmax", name="classification")(x)


In [ ]:
model = Sequential([
    Input(shape=(32,32,3)),
    UpSampling2D(size=(7,7)),
    restnet50_model,
    GlobalAveragePooling2D(),
    Flatten(),
    Dense(1024, activation="relu"),
    Dense(512, activation="relu"),
    Dense(10, activation="softmax", name="classification")

])
model.compile(optimizer='SGD',
                loss='categorical_crossentropy',
                metrics = ['accuracy'])

In [ ]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 up_sampling2d_1 (UpSamplin  (None, 224, 224, 3)       0         
 g2D)                                                            
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_3 (Dense)             (None, 512)               5

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
res_train = preprocess_image_input(X_train)
res_test  = preprocess_image_input(X_test)


res_train_one_hot = to_categorical(Y_train, num_classes=10)
res_test_one_hot = to_categorical(Y_test, num_classes=10)


X, X_CV, Y, Y_CV = train_test_split(res_train, res_train_one_hot,train_size = 0.8, random_state=1)

In [ ]:
print(X.shape)
print(Y.shape)
print(X_CV.shape)
print(Y_CV.shape)

(40000, 32, 32, 3)
(40000, 10)
(10000, 32, 32, 3)
(10000, 10)


In [ ]:
model.fit(X, Y, epochs=10, validation_data = (X_CV, Y_CV), batch_size=32)

Epoch 1/10
1250/1250 [==============================] - 448s 337ms/step - loss: 0.3751 - accuracy: 0.8736 - val_loss: 0.2330 - val_accuracy: 0.9199
Epoch 2/10
1250/1250 [==============================] - 427s 342ms/step - loss: 0.1040 - accuracy: 0.9657 - val_loss: 0.1612 - val_accuracy: 0.9465
Epoch 3/10
1250/1250 [==============================] - 415s 332ms/step - loss: 0.0388 - accuracy: 0.9873 - val_loss: 0.2023 - val_accuracy: 0.9399
Epoch 4/10
1250/1250 [==============================] - 429s 343ms/step - loss: 0.0205 - accuracy: 0.9930 - val_loss: 0.1663 - val_accuracy: 0.9518
Epoch 5/10
1250/1250 [==============================] - 429s 343ms/step - loss: 0.0124 - accuracy: 0.9963 - val_loss: 0.1664 - val_accuracy: 0.9561
Epoch 6/10
1250/1250 [==============================] - 429s 343ms/step - loss: 0.0112 - accuracy: 0.9963 - val_loss: 0.1829 - val_accuracy: 0.9511
Epoch 7/10
1250/1250 [==============================] - 429s 343ms/step - loss: 0.0093 - accuracy: 0.9973 - val_

In [ ]:
rest_test_loss, rest_test_acc = model.evaluate(res_test, res_test_one_hot)
print('\nTest accuracy:', rest_test_acc)

313/313 [==============================] - 30s 95ms/step - loss: 0.1755 - accuracy: 0.9589

Test accuracy: 0.958899974822998
